In [16]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-132652")
exp = Experiment(workspace=ws, name="udacity-project-hyper")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-132652
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-132652


In [17]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "cpu-cluster-2"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing Comput Target")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

### YOUR CODE HERE ###

Found existing Comput Target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
[*range(1,10)]

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [19]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps1 = RandomParameterSampling(
     {"--C": uniform(1,100), "--max_iter": choice(1,5,10,20,30,40,50,80,100,200,400,800,1000)}
)

### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
arguments = {"--C": 1.0, "--max_iter": 100}
current_dir = os.getcwd()
est = SKLearn(source_directory=current_dir, 
              entry_script="train.py",
              compute_target = compute_target,
              script_params = arguments)

### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps1,
    primary_metric_name="Accuracy",
    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=2
)

In [20]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run_hyper = exp.submit(config=hyperdrive_config)
### YOUR CODE HERE ###

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C', '--max_iter'] is the list of overridden parameter(s).


In [ ]:
run_hyper.wait_for_completion(show_output=True)

RunId: HD_f0a86ecf-4d01-4366-95bd-db42947a3798
Web View: https://ml.azure.com/experiments/udacity-project-hyper/runs/HD_f0a86ecf-4d01-4366-95bd-db42947a3798?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-132652/workspaces/quick-starts-ws-132652

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-31T16:18:55.042737][API][INFO]Experiment created<END>\n""<START>[2020-12-31T16:18:55.586476][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n"<START>[2020-12-31T16:18:55.8880388Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-31T16:18:55.757396][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = run_hyper.get_best_run_by_metric()
model = best_run.register_model(model_name="best-hyper-model", "outputs/model")

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", header=True, validate=True)


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.core import Dataset

In [ ]:
Dataset.from_pandas_dataframe(x)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    label_column_name="y",
    n_cross_validations=5)

In [2]:
# Submit your automl run
run_auto = exp.submit(automl_config)
run_auto.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###